In [1]:
# BLP Demand Simulationa and Estimation
# Notation: I use underscore to denote empirical counterparts

# Import packages
using Optim
using Distributions
using Statistics
using DataFrames
using CSV

In [2]:

function demand(p::Vector, X::Matrix, β::Vector, ξ::Matrix, ζ::Matrix)::Tuple{Vector, Number}
    """Compute demand"""
    δ = [X p] * (β .+ ζ)                    # Mean value
    δ0 = zeros(1, size(ζ, 2))               # Mean value of the outside option
    u = [δ; δ0] + ξ                         # Utility
    e = exp.(u)                             # Take exponential
    q = mean(e ./ sum(e, dims=1), dims=2)   # Compute demand
    return q[1:end-1], q[end]
end;

function profits(p::Vector, c::Vector, X::Matrix, β::Vector, ξ::Matrix, ζ::Matrix)::Vector
    """Compute profits"""
    q, _ = demand(p, X, β, ξ, ζ)            # Compute demand
    pr = (p - c) .* q                       # Compute profits
    return pr
end;

function profits_j(pj::Number, j::Int, p::Vector, c::Vector, X::Matrix, β::Vector, ξ::Matrix, ζ::Matrix)::Number
    """Compute profits of firm j"""
    p[j] = pj                               # Insert price of firm j
    pr = profits(p, c, X, β, ξ, ζ)          # Compute profits
    return pr[j]
end;

function equilibrium(c::Vector, X::Matrix, β::Vector, ξ::Matrix, ζ::Matrix)::Vector
    """Compute equilibrium prices and profits"""
    p = 2 .* c;
    dist = 1;
    iter = 0;

    # Iterate until convergence
    while (dist > 1e-8) && (iter<1000)

        # Compute best reply for each firm
        p_old = copy(p);
        for j=1:length(p)
            obj_fun(pj) = - profits_j(pj[1], j, p, c, X, β, ξ, ζ);
            optimize(x -> obj_fun(x), [1.0], LBFGS());
        end

        # Update distance
        dist = max(abs.(p - p_old)...);
        iter += 1;
    end
    return p
end;

function draw_data(I::Int, J::Int, K::Int, rangeJ::Vector, varζ::Number, varX::Number, varξ::Number)::Tuple
    """Draw data for one market"""
    J_ = rand(rangeJ[1]:rangeJ[2])              # Number of firms (products)
    X_ = rand(Exponential(varX), J_, K)         # Product characteristics
    ξ_ = rand(Normal(0, varξ), J_+1, I)         # Product-level utility shocks
    # Consumer-product-level preference shocks
    ζ_ = [rand(Normal(0,1), 1, I) * varζ; zeros(K,I)]
    w_ = rand(Uniform(0, 1), J_)                # Cost shifters
    ω_ = rand(Uniform(0, 1), J_)                # Cost shocks
    c_ = w_ + ω_                                # Cost
    j_ = sort(sample(1:J, J_, replace=false))   # Subset of firms
    return X_, ξ_, ζ_, w_, c_, j_
end;

function compute_mkt_eq(I::Int, J::Int, β::Vector, rangeJ::Vector, varζ::Number, varX::Number, varξ::Number)::DataFrame
    """Compute equilibrium one market"""

    # Initialize variables
    K = size(β, 1) - 1
    X_, ξ_, ζ_, w_, c_, j_ = draw_data(I, J, K, rangeJ, varζ, varX, varξ)

    # Compute equilibrium
    p_ = equilibrium(c_, X_, β, ξ_, ζ_)    # Equilibrium prices
    q_, q0 = demand(p_, X_, β, ξ_, ζ_)     # Demand with shocks
    pr_ = (p_ - c_) .* q_                       # Profits

    # Save to data
    q0_ = ones(length(j_)) .* q0
    df = DataFrame(j=j_, w=w_, p=p_, q=q_, q0=q0_, pr=pr_)
    for k=1:K
      df[!,"x$k"] = X_[:,k]
      df[!,"z$k"] = sum(X_[:,k]) .- X_[:,k]
    end
    return df
end;

function simulate_data(I::Int, J::Int, β::Vector, T::Int, rangeJ::Vector, varζ::Number, varX::Number, varξ::Number)
    """Simulate full dataset"""
    df = compute_mkt_eq(I, J, β, rangeJ, varζ, varX, varξ)
    df[!, "t"] = ones(nrow(df)) * 1
    for t=2:T
        df_temp = compute_mkt_eq(I, J, β, rangeJ, varζ, varX, varξ)
        df_temp[!, "t"] = ones(nrow(df_temp)) * t
        append!(df, df_temp)
    end
    return df
end;

function implied_shares(Xt_::Matrix, ζt_::Matrix, δt_::Vector, δ0::Matrix)::Vector
    """Compute shares implied by deltas and shocks"""
    u = [δt_ .+ (Xt_ * ζt_); δ0]                  # Utility
    e = exp.(u)                                 # Take exponential
    q = mean(e ./ sum(e, dims=1), dims=2)       # Compute demand
    return q[1:end-1]
end;

function inner_loop(qt_::Vector, Xt_::Matrix, ζt_::Matrix)::Vector
    """Solve the inner loop: compute delta, given the shares"""
    δt_ = ones(size(qt_))
    δ0 = zeros(1, size(ζt_, 2))
    dist = 1
    counter = 0
    # Iterate until convergence
    while (dist > 1e-8)
        q = implied_shares(Xt_, ζt_, δt_, δ0)
        δt2_ = δt_ + log.(qt_) - log.(q)
        dist = max(abs.(δt2_ - δt_)...)
        δt_ = δt2_
        counter +=1
    end
    # println("inner_loop iterations: ", counter)
    return δt_
end;

function compute_delta(q_::Vector, X_::Matrix, ζ_::Matrix, T::Vector)::Vector
    """Compute residuals"""
    δ_ = zeros(size(T))

    # Loop over each market
    for t in unique(T)
        qt_ = q_[T.==t]                             # Quantity in market t
        Xt_ = X_[T.==t,:]                           # Characteristics in mkt t
        δ_[T.==t] = inner_loop(qt_, Xt_, ζ_)        # Solve inner loop
    end
    return δ_
end;

function compute_xi(X_::Matrix, IV_::Matrix, δ_::Vector)::Tuple
    """Compute residual, given delta (IV)"""
    β_ = inv(IV_' * X_) * (IV_' * δ_)           # Compute coefficients (IV)
    ξ_ = δ_ - X_ * β_                           # Compute errors
    return ξ_, β_
end;

function GMM(varζ_::Number)::Tuple
    """Compute GMM objective function"""
    δ_ = compute_delta(q_, X_, ζ_ * varζ_, T)   # Compute deltas
    ξ_, β_ = compute_xi(X_, IV_, δ_)            # Compute residuals
    gmm = ξ_' * Z_ * Z_' * ξ_ / length(ξ_)^2    # Compute ortogonality condition
    return gmm, β_
end;





In [3]:

## Main

I = 100;                # Number of consumers
J = 10;                 # Number of firms
K = 2;                  # Product characteristics
T = 10;                # Number of markets
β = [.5, 2, -1];        # Preferences
varζ = 5;               # Variance of the random taste
rangeJ = [2, 6];        # Min and max firms per market
varX = 1;               # Variance of X
varξ = 2;               # Variance of xi


In [4]:

# Simulate
df = simulate_data(I, J, β, T, rangeJ, varζ, varX, varξ);

In [5]:

# Retrieve data
T = Int.(df.t)
X_ = [df.x1 df.x2 df.p]
q_ = df.q
q0_ = df.q0
IV_ = [df.x1 df.x2 df.w]
Z_ = [df.x1 df.x2 df.z1 df.z2]


48×4 Matrix{Float64}:
 0.136123   1.0094     7.08834   2.60822
 0.437869   0.653683   6.7866    2.96394
 1.55785    0.742407   5.66662   2.87522
 1.26334    0.231481   5.96112   3.38614
 1.41633    0.926557   5.80813   2.69107
 2.41295    0.0540955  4.81152   3.56353
 2.05947    0.608494   3.60722   1.96332
 2.26212    0.0447712  3.40456   2.52704
 0.0865222  0.695682   5.58016   1.87613
 0.721584   0.588972   4.9451    1.98284
 ⋮                               
 0.556981   0.432378   9.33861   2.73994
 0.454266   1.04435    0.615432  1.34654
 0.615432   1.34654    0.454266  1.04435
 0.445178   0.487536   3.18006   5.64809
 0.611266   0.388912   3.01397   5.74672
 0.856061   0.0378307  2.76918   6.0978
 0.74632    0.0216336  2.87892   6.114
 0.304626   2.54375    3.32062   3.59188
 0.661791   2.65597    2.96345   3.47966

In [10]:

# # Compute logit estimate
# y = log.(df.q) - log.(df.q0)
# β_logit = inv(IV_' * X_) * (IV_' * y)
# print("Estimated logit coefficients: $β_logit")

# Draw shocks (less)
ζ_ = [rand(Normal(0,1), 1, I); zeros(K, I)]

# Minimize GMM objective function
varζ_ = optimize(x -> GMM(x[1])[1], [2.0], LBFGS()).minimizer[1]
println("varζ_: ", varζ_)
β_blp = GMM(varζ_)[2]
print("Estimated BLP coefficients: $β_blp")

varζ_: 7.561361403638421
Estimated BLP coefficients: [2.040441765565401, 2.1241519865313467, -1.2598393639833023]

In [7]:
# ivan li example in python
def objective(theta_2, s, X, V, p, y, J, T, N, marks, markets, tol, 
              Z, Z_s, W, weigh, firms):
    
    obs = np.sum(J)
    
    # Aviv's step 1 & 2:
    d.delta = solve_delta(s, X, V, p, y, d.delta, theta_2, J, T, N, tol)
    
    # obtain the actual implied quantities and shares from converged delta
    q_s = compute_share(X, V, p, y, d.delta, theta_2, J, T, N)
    
    # calculate marginal costs
    mc = calc_mc(q_s, firms, p, y, theta_2[5], J, T, N, marks, markets).reshape((-1,1))
    
    # since we are using both demand and supply side variables,
    #  we want to stack the estimated delta and estimated mc
    y2 = np.vstack((d.delta.reshape((-1,1)), np.log(mc)))
    
    # create characteristics matrix that includes both supply and demand side
    #  with demand characteristics on the bottom left and supply on the top right
    x = scipy.linalg.block_diag(X,W)
    
    # create matrix of supply and demand instruments, again with
    #  demand instruments on the right and supply on the left (top/down changed)    
    z = scipy.linalg.block_diag(Z,Z_s)
    
    # get linear parameters (this FOC is from Aviv's appendix)
    b = np.linalg.inv(x.T @ z @ weigh @ z.T @ x) @ (x.T @ z @ weigh @ z.T @ y2)
    
    # Step 3: get the error term xi (also called omega)
    xi_w = y2 - x @ b
    
    # computeo g_bar in GMM methods
    g = z.T @ xi_w / obs
    
    obj = float(obs**2 * g.T @ weigh @ g)
   
    print([theta_2, obj])
    
    return obj